# Estimating the aDDM

Author: Brenden Eum (2023)

This script will read already-transformed data containing trial-by-trial details and fixations and fit the aDDM to this data. It uses the Tavares Toolbox (Tavares et al. 2017) rewritten for Julia by Lynn in Summer 2023. See https://github.com/aDDM-Toolbox/ADDM.jl for the toolbox. 

I've made a changes to the toolbox in order to suit my project. Some of these are small changes, like changing object types, saving outputs to csv files, or changing the grid for grid search. I don't document small changes since they'll probably change over multiple iterations. I do document major changes to the toolbox: (1) The toolbox now fits the model to the output of load_data_from_csv instead of simulating data; (2) The toolbox no longer throws a domain error when $\theta \notin [0,1]$.

Inputs for this notebook:
- Data
    - data/expdata*.csv:    Data pertaining to the details of each trial. An observation is a subject-trial. It must include the following columns [parcode,trial,rt,choice,item_left,item_right], which correspond to [subject, trial, response time in ms, choice (-1 left, 1 right), value of left item, value of right item] respectively.
    - data/fixations*.csv:  Data pertaining to the fixations within each trial. An observation is a subject-trial-fixation. It must include columns [parcode,trial,fix_item,fix_time], which correspond to [subject, trial, fixation location (1 left, 2 right, otherwise other), fixation duration in ms].
- Code
    - src/ addm_grid_search.jl: Takes a grid of parameters, makes every combination, and calculates negative log likelihood (NLL) of aDDM for every combo given some data.
    - addm.jl: Defines aDDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - ddm.jl: Defines DDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - util.jl: Contains functions for prepping data for aDDM fitting and estimating empirical fixation properties.


Outputs for this notebook:
- GainFit.csv:  aDDM estimates for the gain condition. Estimates are for subjects included in the input datasets.
- LossFit.csv:  aDDM estimates for the loss condition.

# Activate the Environment

Typically this happens by

```
using Pkg
Pkg.activate("addm")
```

**Note that single vs. double quotes matter.**

But this does not work as expected in a notebook unless the `Project.toml` is in the same folder as the notebook file.

In [1]:
pwd();
Base.load_path();
include("../../src/addm_grid_search.jl");
Base.load_path();

  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


# Preamble

A set of standard things I like to do before getting started with any notebook.

In [2]:
import Pkg;
#Pkg.add("DataFrames");
#Pkg.add("CSV");
using DataFrames;
using CSV;

# Common Model Settings

In [3]:
grid_search_terminate_threshold = .0001 # %change in total NLL that needs to be achieved to terminate the iterative grid searchh = .1%

0.0001

# Function: Make new grid

In [ ]:
function make_new_grid(estimates::DataFrame, data::Dict{String, Vector{aDDMTrial}}, dStepSize::Number, σStepSize::Number, θStepSize::Number, bStepSize::Number, iteration::Number; bounded_theta::Bool)
    
    dGrid = Any[]
    σGrid = Any[]
    θGrid = Any[]
    bGrid = Any[]

    iteration = iteration-1 # Iteration in the while loops below start at 2 since we run the fitting once before the loop. Correct for this.
    ind = 1
    for subject in collect(keys(data))

        d = estimates[(estimates.subject .== subject), "d"][1]
        σ = estimates[(estimates.subject .== subject), "s"][1]
        θ = estimates[(estimates.subject .== subject), "t"][1]
        b = estimates[(estimates.subject .== subject), "b"][1]

        dLow = max(0.00001, d-(dStepSize/(2^iteration)));
        dHigh = d+(dStepSize/(2^iteration));
        σLow = max(0.001, σ-(σStepSize/(2^iteration)));
        σHigh = σ+(σStepSize/(2^iteration));
        if !bounded_theta
            θLow = θ-(θStepSize/(2^iteration));
            θHigh = θ+(θStepSize/(2^iteration));
        else
            θLow = max(0, θ-(θStepSize/(2^iteration)));
            θHigh = min(1, θ+(θStepSize/(2^iteration)));
        end
        bLow = max(-.99, b-(bStepSize/(2^iteration)));
        bHigh = min(.99, b+(bStepSize/(2^iteration)));

        push!(dGrid, float([dLow, d, dHigh]))
        push!(σGrid, float([σLow, σ, σHigh]))    
        push!(θGrid, float([θLow, θ, θHigh]))           
        push!(bGrid, float([bLow, b, bHigh]))  

        ind += 1
    end
    return dGrid, σGrid, θGrid, bGrid
end

# aDDM

Let's do a 2-iteration grid search. We'll start with a low resolution grid search, then once we have rough estimates per subject, we can zoom into the region around the rough estimates for a precise measure.

## Prep for gain and loss: load data, placeholders, set step sizes

In [ ]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .0025 # Step sizes for the grids.
σStepSize = .03
θStepSize = .5
bStepSize = .1

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.008;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([0:θStepSize:2;]))    # θ from 0 to 2
    push!(bInitialGrid, [-.2:bStepSize:.2;])      # b from -.2 to .2
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])

## Function: Grid search and report data frame

In [ ]:
function fit_aDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = aDDM_grid_search(addm, data, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    return df, NLLsList
end

## Fit Gain Data

In [ ]:
oldGainEstimates, oldNLLs = fit_aDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newGainEstimates, newNLLs = fit_aDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates = oldGainEstimates
        global NLL = oldNLLs
    else
        global gainEstimates = newGainEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL;

end
CSV.write("NumericGainFit.csv", gainEstimates)
CSV.write("NumericGainNLL.csv", Tables.table(NLL), writeheader=false)

## Fit Loss Data

In [ ]:
oldLossEstimates, oldNLLs = fit_aDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newLossEstimates, newNLLs = fit_aDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit.csv", lossEstimates)
CSV.write("NumericLossNLL.csv", Tables.table(NLL), writeheader=false)

## Fit Loss Data with Bounded $\theta$

We know this is not good, but it would be nice to see the results of simulations using theta bounded between 0 and 1.

In [ ]:
θBoundedGrid = Any[]
for subject in collect(keys(dataLoss))
    push!(θBoundedGrid, float([0:θStepSize:1;]))    # θ from 0 to 1
end
oldLossEstimates, oldNLLs = fit_aDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θBoundedGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=true)
    newLossEstimates, newNLLs = fit_aDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates_boundedTheta = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates_boundedTheta = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates_boundedTheta); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit_boundedTheta.csv", lossEstimates_boundedTheta)
CSV.write("NumericLossNLL_boundedTheta.csv", Tables.table(NLL), writeheader=false)

# Additive Model (addDDM)

## Prep

In [ ]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .0025 # Step sizes for the grids.
σStepSize = .03
θStepSize = 1
bStepSize = .1

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.008;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([-3:θStepSize:3;]))    # θ from -3 to 3
    push!(bInitialGrid, [-.2:bStepSize:.2;])      # b from -.25 to .25
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])

## Function: Grid search and record

In [ ]:
function fit_addDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = addDDM_grid_search(addm, data, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    return df, NLLsList
end

## Fit Gain Data

In [ ]:
oldGainEstimates, oldNLLs = fit_addDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newGainEstimates, newNLLs = fit_addDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates_Add = oldGainEstimates
        global NLL = oldNLLs
    else
        global gainEstimates_Add = newGainEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates_Add); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL;

end
CSV.write("NumericGainFit_Add.csv", gainEstimates_Add)
CSV.write("NumericGainNLL_Add.csv", Tables.table(NLL), writeheader=false)

## Fit Loss Data

In [ ]:
oldLossEstimates, oldNLLs = fit_addDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newLossEstimates, newNLLs = fit_addDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates_Add = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates_Add = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates_Add); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit_Add.csv", lossEstimates_Add)
CSV.write("NumericLossNLL_Add.csv", Tables.table(NLL), writeheader=false)

# Range Normalized Model (RNaDDM)

## Prep

In [ ]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .005 # Step sizes for the grids.
σStepSize = .03
θStepSize = .25

bStepSize = .1

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.0401;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([0:θStepSize:2;]))    # θ from 0 to 2
    push!(bInitialGrid, [-.2:bStepSize:.2;])      # b from -.2 to .2
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])

## Function: Grid search and results

In [ ]:
function fit_RNaDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number; minValue::Number=-6, maxValue::Number=-1)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = RNaDDM_grid_search(addm, data, minValue, maxValue, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    return df, NLLsList
end

## Fit Gain Data

In [ ]:
oldGainEstimates, oldNLLs = fit_RNaDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount; minValue=1, maxValue=6)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newGainEstimates, newNLLs = fit_RNaDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount; minValue=1, maxValue=6)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates_RN = oldGainEstimates
        global NLL = oldNLLs
    else
        global gainEstimates_RN = newGainEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates_RN); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL; 

end
CSV.write("NumericGainFit_RN.csv", gainEstimates_RN)
CSV.write("NumericGainNLL_RN.csv", Tables.table(NLL), writeheader=false)

## Fit Loss Data

In [ ]:
oldLossEstimates, oldNLLs = fit_RNaDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newLossEstimates, newNLLs = fit_RNaDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates_RN = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates_RN = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates_RN); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit_RN.csv", lossEstimates_RN)
CSV.write("NumericLossNLL_RN.csv", Tables.table(NLL), writeheader=false)

# Divisive Normalized Model (DNaDDM)

## Prep

In [ ]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .005 # Step sizes for the grids.
σStepSize = .03
θStepSize = .25

bStepSize = .1

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.0401;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([0:θStepSize:2;]))    # θ from 0 to 2
    push!(bInitialGrid, [-.2:bStepSize:.2;])      # b from -.2 to .2
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])

## Function: Grid search and results

In [ ]:
function fit_DNaDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = DNaDDM_grid_search(addm, data, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    return df, NLLsList
end

## Fit Gain Data

In [ ]:
oldGainEstimates, oldNLLs = fit_DNaDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newGainEstimates, newNLLs = fit_DNaDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates_DN = oldGainEstimates
        global NLL = oldNLLs
    else
        global gainEstimates_DN = newGainEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates_DN); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL; 

end
CSV.write("NumericGainFit_DN.csv", gainEstimates_DN)
CSV.write("NumericGainNLL_DN.csv", Tables.table(NLL), writeheader=false)

## Fit Loss Data

In [ ]:
oldLossEstimates, oldNLLs = fit_DNaDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration; bounded_theta=false)
    newLossEstimates, newNLLs = fit_DNaDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates_DN = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates_DN = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates_DN); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit_DN.csv", lossEstimates_DN)
CSV.write("NumericLossNLL_DN.csv", Tables.table(NLL), writeheader=false)

# Range Normalized Plus Model (RNPaDDM)

Range normalized, but with an additional constant that shifts the lower bound on the range.

## Prep

In [4]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .005 # Step sizes for the grids.
σStepSize = .03
θStepSize = .25
bStepSize = .1
kStepSize = .5

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
kInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.0401;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([0:θStepSize:2;]))    # θ from 0 to 2
    push!(bInitialGrid, [-.2:bStepSize:.2;])      # b from -.2 to .2
    push!(kInitialGrid, [0:kStepSize:2;])      # k from 0 to 2
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])
println(kInitialGrid[1])

[0.0001, 0.0051, 0.0101, 0.0151, 0.0201, 0.0251, 0.0301, 0.0351, 0.0401]
[0.01, 0.04, 0.07, 0.1]
[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
[-0.2, -0.1, 0.0, 0.1, 0.2]
[0.0, 0.5, 1.0, 1.5, 2.0]


## Function: Make new grid with k

In [5]:
function make_new_grid_RNP(estimates::DataFrame, data::Dict{String, Vector{aDDMTrial}}, dStepSize::Number, σStepSize::Number, θStepSize::Number, bStepSize::Number, kStepSize::Number, iteration::Number; bounded_theta::Bool)
    
    dGrid = Any[]
    σGrid = Any[]
    θGrid = Any[]
    bGrid = Any[]
    kGrid = Any[]

    iteration = iteration-1 # Iteration in the while loops below start at 2 since we run the fitting once before the loop. Correct for this.
    ind = 1
    for subject in collect(keys(data))

        d = estimates[(estimates.subject .== subject), "d"][1]
        σ = estimates[(estimates.subject .== subject), "s"][1]
        θ = estimates[(estimates.subject .== subject), "t"][1]
        b = estimates[(estimates.subject .== subject), "b"][1]
        k = estimates[(estimates.subject .== subject), "k"][1]

        dLow = max(0.00001, d-(dStepSize/(2^iteration)));
        dHigh = d+(dStepSize/(2^iteration));
        σLow = max(0.001, σ-(σStepSize/(2^iteration)));
        σHigh = σ+(σStepSize/(2^iteration));
        if !bounded_theta
            θLow = θ-(θStepSize/(2^iteration));
            θHigh = θ+(θStepSize/(2^iteration));
        else
            θLow = max(0, θ-(θStepSize/(2^iteration)));
            θHigh = min(1, θ+(θStepSize/(2^iteration)));
        end
        bLow = max(-.99, b-(bStepSize/(2^iteration)));
        bHigh = min(.99, b+(bStepSize/(2^iteration)));
        kLow = max(0, k-(kStepSize/(2^iteration)));
        kHigh = k+(kStepSize/(2^iteration));

        push!(dGrid, float([dLow, d, dHigh]))
        push!(σGrid, float([σLow, σ, σHigh]))    
        push!(θGrid, float([θLow, θ, θHigh]))           
        push!(bGrid, float([bLow, b, bHigh]))  
        push!(kGrid, float([kLow, k, kHigh]))  

        ind += 1
    end
    return dGrid, σGrid, θGrid, bGrid, kGrid
end

make_new_grid_RNP (generic function with 1 method)

## Grid search and results

In [6]:
function fit_RNPaDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, kGrid::Vector{Any}, subjectCount::Number; minValue::Number=-6, maxValue::Number=-1)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    kList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, kEst, NLL_Indiv = RNPaDDM_grid_search(addm, data, minValue, maxValue, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], kGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        kList[ind] = kEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList,
        k = kList
    )
    df = sort(df, :subject)
    return df, NLLsList
end

fit_RNPaDDM_custom_resolution (generic function with 1 method)

## Fit gain data

In [7]:
oldGainEstimates, oldNLLs = fit_RNPaDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, kInitialGrid, subjectCount; minValue=1, maxValue=6)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid, kGrid = make_new_grid_RNP(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, kStepSize, iteration; bounded_theta=false)
    newGainEstimates, newNLLs = fit_RNPaDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, kGrid, subjectCount; minValue=1, maxValue=6)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates_RNP = oldGainEstimates
        global NLL = oldNLLs
    else
        global gainEstimates_RNP = newGainEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates_RNP); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL; 

end
CSV.write("NumericGainFit_RNP.csv", gainEstimates_RNP)
CSV.write("NumericGainNLL_RNP.csv", Tables.table(NLL), writeheader=false)

Progress:   8%|███▏                                     |  ETA: 3:03:55

Progress:  12%|████▊                                    |  ETA: 3:07:19

Progress:  15%|██████▎                                  |  ETA: 3:21:50

Progress:  19%|███████▉                                 |  ETA: 3:10:10

Progress:  23%|█████████▌                               |  ETA: 2:57:40

...

Progress:  92%|█████████████████████████████████████▉   |  ETA: 0:00:29

Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:00:14

Progress: 100%|█████████████████████████████████████████| Time: 0:06:13


Iteration 7
Old NLL 13057.553657394437
New NLL 13057.139993355577
Percent Change 3.168005659515955e-5
26×6 DataFrame
 Row │ subject  d           s          t         b           k         
     │ String   Float64     Float64    Float64   Float64     Float64   
─────┼─────────────────────────────────────────────────────────────────
   1 │ 201      0.0109594   0.0503125  0.824219  -0.046875   1.24219
   2 │ 202      0.0126781   0.0428125  0.765625   0.0265625  1.10156
   3 │ 204      0.0126781   0.0451562  0.808594  -0.046875   0.03125
   4 │ 205      0.00908438  0.0615625  0.777344  -0.046875   1.74219
   5 │ 206      0.0201      0.060625   0.773438  -0.046875   0.0546875
   6 │ 207      0.00783437  0.0446875  0.621094  -0.046875   0.117188
   7 │ 208      0.0237719   0.0732813  0.816406  -0.046875   0.0078125
   8 │ 209      0.00814687  0.0465625  0.882812  -0.046875   0.992188
   9 │ 211      0.0126781   0.041875   0.714844  -0.046875   0.148438
  10 │ 213      0.0115844   0.0845313  

"NumericGainNLL_RNP.csv"

## Fit loss data

In [8]:
oldLossEstimates, oldNLLs = fit_RNPaDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, kInitialGrid, subjectCount)
oldNLL = sum(oldNLLs)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > grid_search_terminate_threshold
    
    dGrid, σGrid, θGrid, bGrid, kGrid = make_new_grid_RNP(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, kStepSize, iteration; bounded_theta=false)
    newLossEstimates, newNLLs = fit_RNPaDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, kGrid, subjectCount)
    newNLL = sum(newNLLs)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates_RNP = oldLossEstimates
        global NLL = oldNLLs
    else
        global lossEstimates_RNP = newLossEstimates
        global NLL = newNLLs
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates_RNP); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("NumericLossFit_RNP.csv", lossEstimates_RNP)
CSV.write("NumericLossNLL_RNP.csv", Tables.table(NLL), writeheader=false)

Progress:   8%|███▏                                     |  ETA: 4:17:22

Progress:  12%|████▊                                    |  ETA: 4:15:36

Progress:  15%|██████▎                                  |  ETA: 4:15:52

Progress:  19%|███████▉                                 |  ETA: 3:51:42

Progress:  23%|█████████▌                               |  ETA: 3:28:08

...

Progress:  92%|█████████████████████████████████████▉   |  ETA: 0:00:32

Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:00:16

Progress: 100%|█████████████████████████████████████████| Time: 0:06:52


Iteration 7
Old NLL 13567.96337876253
New NLL 13567.39973970094
Percent Change 4.154190616930568e-5
26×6 DataFrame
 Row │ subject  d           s          t            b           k         
     │ String   Float64     Float64    Float64      Float64     Float64   
─────┼────────────────────────────────────────────────────────────────────
   1 │ 201      0.00635     0.05125     0.78125     -0.046875   2.28125
   2 │ 202      0.0158031   0.0404687   0.457031     0.0484375  0.0390625
   3 │ 204      0.0126781   0.0446875   0.699219     0.0015625  0.0
   4 │ 205      0.003225    0.0648438  -0.246094    -0.046875   1.86719
   5 │ 206      0.0137719   0.0484375   0.855469     0.0484375  0.179688
   6 │ 207      0.00455312  0.0446875   1.19141     -0.046875   0.0
   7 │ 208      0.0155687   0.0723437   0.777344    -0.046875   2.24219
   8 │ 209      0.00892812  0.041875    0.433594    -0.046875   0.242188
   9 │ 211      0.0136156   0.0404688   0.402344    -0.046875   0.0
  10 │ 213      0.01

"NumericLossNLL_RNP.csv"